In [13]:
from pycocotools.coco import COCO
import nltk
from data_loader import get_loader
from torchvision import transforms
from pathlib import Path
import json

## files

First of all let's look at files that we have. We have (as we know) around `80K` training examples.

In [3]:
data_dir = Path.home() / 'data'

In [5]:
list(data_dir.glob('*'))

[PosixPath('/home/ilyarudyak/data/annotations'),
 PosixPath('/home/ilyarudyak/data/images')]

### images

In [7]:
list(data_dir.glob('images/*'))

[PosixPath('/home/ilyarudyak/data/images/train2014'),
 PosixPath('/home/ilyarudyak/data/images/test2014'),
 PosixPath('/home/ilyarudyak/data/images/val2014')]

In [8]:
len(list(data_dir.glob('images/train2014/*')))

82783

### captions

So we have about `5` captions for each image. They are stored in an arbitrary order.

In [10]:
list(data_dir.glob('annotations/captions*'))

[PosixPath('/home/ilyarudyak/data/annotations/captions_train2014.json'),
 PosixPath('/home/ilyarudyak/data/annotations/captions_val2014.json')]

In [14]:
data = json.load(open(data_dir / 'annotations/captions_train2014.json')) 

In [15]:
data.keys()

dict_keys(['info', 'images', 'licenses', 'annotations'])

In [18]:
len(data['images'])

82783

In [25]:
data['images'][0]

{'license': 5,
 'file_name': 'COCO_train2014_000000057870.jpg',
 'coco_url': 'http://images.cocodataset.org/train2014/COCO_train2014_000000057870.jpg',
 'height': 480,
 'width': 640,
 'date_captured': '2013-11-14 16:28:13',
 'flickr_url': 'http://farm4.staticflickr.com/3153/2970773875_164f0c0b83_z.jpg',
 'id': 57870}

In [19]:
len(data['annotations'])

414113

In [20]:
414113 / 82783

5.0023917954169335

In [26]:
data['annotations'][0]

{'image_id': 318556,
 'id': 48,
 'caption': 'A very clean and well decorated empty bathroom'}